In [1]:
import pandas as pd
import re
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [9]:
df_orig = pd.read_csv("gaon/gaon_monthly_chart_until_2009.csv")
df_orig.head()

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,num_times_top_1
0,#첫사랑,볼빨간사춘기,#첫사랑,"쇼파르뮤직, 딩고뮤직",카카오 M,201801-201802-201803,1,0,1,1,0
1,#화이트 스타일 (Feat. 김호연 of 달 좋은밤),키썸 (Kisum),Show Off,스케치,카카오 M,201507,0,0,0,0,0
2,%%(응응),에이핑크 (Apink),PERCENT,플레이엠엔터테인먼트,카카오 M,201901-201902,1,0,0,0,0
3,00_00 (Zero O`Clock),방탄소년단,MAP OF THE SOUL _ 7,빅히트 엔터테인먼트,Dreamus,202003-202004-202005,1,0,0,0,0
4,0330,유키스 (U-Kiss),Bran New Kiss (EP),NH EMG,카카오 M,201105,0,0,0,0,0


In [10]:
df = df_orig.copy()

In [11]:
df.dtypes

title               object
singer              object
album               object
producer            object
distributor         object
year_month          object
is_popular           int64
external_factors     int64
num_times_top_10     int64
num_times_top_5      int64
num_times_top_1      int64
dtype: object

In [12]:
df.shape

(3629, 11)

Los textos se ponen en minúscula.

In [13]:
def df_to_lowercase(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.lower()

In [14]:
df_to_lowercase(df)

Se borran las canciones cuyos artistas sean internacionales

In [15]:
international_singers = ['new hope club, danna paola', 'dua lipa', 'david guetta & showtek', 'eye to eye', 'pharrell williams', 'justin hurwitz', 'r. kelly', 'etham', 'madonna', 'craig david', 'bad meets evil', 'josh gad', 'naomi scott', 'maia wilson', 'sweetbox', 'demi lovato', 'karina', 'sarah brightman', 'lauv', 'marshmello, anne-marie', 'arizona zervas', 'jonathan groff', 'anne hathaway', 'la la land cast', 'jessie j, ariana grande, nicki minaj', 'redfoo (lmfao)', 'ruel', 'idina menzel', 'chuckie & lmfao', 'david guetta', 'lauv, anne-marie', 'les miserables cast', 'cessyl orchestra', 'major lazer', 'billie eilish', 'carla bruni', 
                         'cast - frozen', 'samantha barks', 'mena massoud, naomi scott', 'flo rida', 'kristen bell, idina menzel, josh gad, jonathan groff, cast of frozen 2', 'jay-z & kanye west', 'lizzo', 'will.i.am', 'ed sheeran, justin bieber', 'justin bieber', 'ed sheeran', 'shawn mendes', 'jason derulo', 'shawn mendes, camila cabello', 'the beatles', 'kristen bell, santino fontana', 
 'sara bareilles', 'ellie goulding', 'conan gray', 'hozier', 'selena gomez', 'straight no chaser', 'troye sivan', 'daft punk', 'mark ronson', 'keala settle, the greatest showman ensemble', 'the chainsmokers', 'twin forks', 'ylvis', 'britney spears', 'fitz & the tantrums', 'evan rachel wood', 'rachael yamagata', 'caro emerald', 'katy perry', 'coldplay', 'mariah carey', 'ryan gosling & emma stone', 
                         'charlie puth', 'sia', 'eminem', 'adele', 'carly rae jepsen', 'anne-marie', 'imagine dragons', 'brian mcknight', 
                         'it`s', 'alicia keys', 'queen', 'jeremy zucker', 'lady gaga', 'tones and i', 'charlie puth, wiz khalifa', 
                         'camila cabello', 'taylor swift', 'avril lavigne', 'jeff bernat', 'bruno mars', 'will smith', 'jason mraz',
                        'sam smith', 'kristen bell', "michael jackson", 'justin timberlake', 'adam levine', 'keira knightley', 'maroon 5',
                        'kelly clarkson', 'zara larsson', 'lauv, troye sivan',
                         'marshmello & anne-marie', 'post malone & swae lee', 'yolanda be cool & dcup']
df = df[~df.singer.isin(international_singers)]
# Tras explorar los datos se ve como los siguientes artistas no hacen dúos con artistas coreanos
df = df[~df.singer.str.contains("justin bieber")]
df = df[~df.singer.str.contains("ariana grande")]
df = df[~df.singer.str.contains("chainsmokers")]
df = df[~df.singer.str.contains("idina menzel")]

In [16]:
df.shape

(3502, 11)

--------------------

## DBKpop tablas

Se importan los datos obtenidos de la página "DBKpop". Se visualizan algunos registros, se eliminan los duplicados y se introducen algunos artistas que no estaban pero que se encuentran en las canciones obtenidas de *Gaon*.

In [17]:
df_girl = pd.read_csv("dbkpop/all_k_pop_girlgroups.csv")
df_idols = pd.read_csv("dbkpop/all_k_pop_idols.csv")
df_boy = pd.read_csv("dbkpop/all_kpop_boybands.csv")

In [18]:
df_girl.head()

,profile,name,short,korean_name,debut,company,members,orig._memb.,fanclub_name,active
0,NaN,Girls' Generation,SNSD,소녀시대,2007-08-05,SM,5,9,SONE,Yes
1,NaN,KARA,NaN,카라,2007-03-29,DSP,4,5,Kamilia,No
2,NaN,2NE1,NaN,투애니원,2009-05-06,YG,4,4,Blackjack,No
3,Profile,T-ara,NaN,티아라,2009-07-29,MBK,4,6,QUEEN'S,Hiatus
4,Profile,SISTAR,NaN,씨스타,2010-03-06,Starship,4,4,STAR1,No


In [19]:
df_girl["gender"] = "girlgroup"
df_girl.drop_duplicates(keep='first', inplace=True)

Se añaden algunos grupos encontrados en las canciones pero no en las tablas.

In [20]:
df_boy.head()

,profile,name,short,korean_name,debut,company,members,orig._memb.,fanclub_name,active
0,NaN,BIGBANG,NaN,빅뱅,2006-08-19,YG,4,5,VIP,Yes
1,Profile,Super Junior,SJ,슈퍼주니어,2005-11-06,SM,11,12,ELF,Yes
2,NaN,TVXQ!,TVXQ,동방신기,2003-12-26,SM,2,5,Cassiopeia,Yes
3,NaN,SHINee,NaN,샤이니,2008-05-22,SM,4,5,Shawol,Yes
4,Profile,EXO,NaN,엑소,2012-04-08,SM,9,12,EXO-L,Yes


In [21]:
def add_boyband(df, name, short, korean_name):
    new_row = [np.nan]*df.shape[1]
    new_row[1] = name
    new_row[2] = short
    new_row[3] = korean_name
    df.loc[-1] = new_row # adding a row
    df.index = df.index + 1  # shifting index
    df = df.sort_index() 

In [22]:
add_boyband(df_boy, name="gb9", short=None, korean_name="길구봉구")
add_boyband(df_boy, name="boys under the moon", short=None, korean_name="월하소년")

In [23]:
df_boy["gender"] = "boyband"

In [24]:
df_boy.head()

,profile,name,short,korean_name,debut,company,members,orig._memb.,fanclub_name,active,gender
2,NaN,BIGBANG,NaN,빅뱅,2006-08-19,YG,4.0,5.0,VIP,Yes,boyband
3,Profile,Super Junior,SJ,슈퍼주니어,2005-11-06,SM,11.0,12.0,ELF,Yes,boyband
4,NaN,TVXQ!,TVXQ,동방신기,2003-12-26,SM,2.0,5.0,Cassiopeia,Yes,boyband
5,NaN,SHINee,NaN,샤이니,2008-05-22,SM,4.0,5.0,Shawol,Yes,boyband
6,Profile,EXO,NaN,엑소,2012-04-08,SM,9.0,12.0,EXO-L,Yes,boyband


In [25]:
df_idols.head()

,profile,stage_name,full_name,korean_name,k._stage_name,date_of_birth,group,country,second_country,height,weight,birthplace,other_group,former_group,gender,position,instagram,twitter
0,Profile,Taeyeon,Kim Taeyeon,김태연,태연,1989-03-09,SNSD,South Korea,NaN,160,44,Jeonju,"TTS, Oh!GG",NaN,F,"Leader, Main Vocalist",taeyeon_ss,NaN
1,NaN,Sunny,Lee Sunkyu,이순규,써니,1989-05-15,SNSD,South Korea,NaN,158,43,California,NaN,NaN,F,NaN,svnnynight,NaN
2,NaN,Tiffany,Hwang Miyoung,황미영,티파니,1989-08-01,SNSD,South Korea,USA,163,50,San Francisco,TTS,NaN,F,NaN,xolovestephi,NaN
3,NaN,Hyoyeon,Kim Hyoyeon,김효연,효연,1989-09-22,SNSD,South Korea,NaN,158,48,Incheon,NaN,NaN,F,NaN,watasiwahyo,NaN
4,NaN,Yuri,Kwon Yuri,권유리,유리,1989-12-05,SNSD,South Korea,NaN,167,45,Goyang,NaN,NaN,F,NaN,yulyulk,NaN


In [26]:
def add_idol(df, stage_name, full_name, korean_name, k_stage_name, gender):
    new_row = [np.nan]*18
    new_row[3] = korean_name
    new_row[4] = k_stage_name
    new_row[1] = stage_name
    new_row[2] = full_name
    new_row[14] = gender
    df.loc[-1] = new_row # adding a row
    df.index = df.index + 1  # shifting index
    df = df.sort_index() 

In [27]:
add_idol(df_idols, stage_name="gill", full_name="lee sang", korean_name="리쌍", k_stage_name="길", gender="m")
add_idol(df_idols, stage_name="40", full_name=None, korean_name="김한준", k_stage_name="포티", gender="m")
df_idols.drop_duplicates(keep='first', inplace=True)

Los textos se ponen en minúscula.

In [29]:
df_to_lowercase(df_girl)
df_to_lowercase(df_idols)
df_to_lowercase(df_boy)

--------------------------

## Explorar 'title', 'album' y 'singer'

In [30]:
import nltk
nltk.download('punkt')
  
from nltk import word_tokenize
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Paula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
words = df['singer'].str.lower().str.cat(sep=' ')
word_tokens = word_tokenize(words)

word_counter = Counter(word_tokens)
print('{} different words'.format(len(word_counter)))
word_counter.most_common(538)

1461 different words


[('(', 1779),
 (')', 1779),
 (',', 403),
 ('아이유', 73),
 ('iu', 72),
 ('방탄소년단', 63),
 ('exo', 61),
 ('버스커', 56),
 ('busker', 56),
 ('다비치', 39),
 ('빅뱅', 38),
 ('one', 38),
 ('the', 38),
 ('악동뮤지션', 37),
 ('akmu', 37),
 ('&', 37),
 ('브라운', 36),
 ('아이드', 36),
 ('원', 36),
 ('소녀시대', 31),
 ('비스트', 30),
 ('허각', 30),
 ('태연', 29),
 ('beast', 29),
 ('백지영', 28),
 ('볼빨간사춘기', 27),
 ('10cm', 27),
 ('taeyeon', 27),
 ('샤이니', 26),
 ('shinee', 26),
 ('2am', 26),
 ('`', 26),
 ('케이윌', 26),
 ('k.will', 26),
 ('에일리', 25),
 ('다이나믹', 25),
 ('듀오', 25),
 ('2ne1', 25),
 ('dynamic', 24),
 ('duo', 24),
 ('티아라', 24),
 ('b', 24),
 ('포맨', 24),
 ('4men', 24),
 ('girl', 23),
 ('투애니', 23),
 ('x', 22),
 ('레드벨벳', 22),
 ('zico', 22),
 ('에픽하이', 22),
 ('거미', 22),
 ('에이핑크', 21),
 ('apink', 21),
 ('girls', 21),
 ('red', 21),
 ('velvet', 21),
 ('crush', 21),
 ('지코', 21),
 ('wan', 21),
 ('na', 21),
 ('워너원', 21),
 ('epik', 21),
 ('high', 21),
 ('mc몽', 21),
 ('크러쉬', 20),
 ('소울', 20),
 ('헤이즈', 20),
 ('리쌍', 20),
 ('씨스타', 20),
 ('지아', 

In [32]:
words = df['title'].str.lower().str.cat(sep=' ')
word_tokens = word_tokenize(words)

word_counter = Counter(word_tokens)
print('{} different words'.format(len(word_counter)))
word_counter.most_common(538)

4488 different words


[('(', 1272),
 (')', 1222),
 ('.', 647),
 ('feat', 545),
 (',', 136),
 ('love', 118),
 ('you', 108),
 ('i', 90),
 ('`', 84),
 ('of', 76),
 ('...', 64),
 ('me', 56),
 ('prod', 55),
 ('&', 53),
 ('my', 46),
 ('with', 45),
 ('the', 42),
 ('너', 37),
 ('by', 33),
 ('내', 33),
 ('u', 32),
 ('밤', 31),
 ('!', 31),
 ('a', 30),
 ('사랑', 29),
 ('it', 28),
 ('?', 27),
 ('사람', 27),
 ('t', 26),
 ('그', 26),
 ('baby', 25),
 ('날', 25),
 ('우리', 25),
 ('in', 24),
 ('day', 24),
 ('girl', 24),
 ('이', 24),
 ('ver', 23),
 ('사랑은', 23),
 ('수', 21),
 ('to', 20),
 ('up', 20),
 ('내가', 20),
 ('don', 19),
 ('나', 19),
 ('너를', 19),
 ('사랑이', 19),
 ('be', 18),
 ('is', 18),
 ('good', 18),
 ('boy', 18),
 ('s', 18),
 ('m', 17),
 ('니가', 17),
 ('one', 16),
 ('나의', 16),
 ('그대', 16),
 ('너에게', 16),
 ('너의', 16),
 ('왜', 16),
 ('for', 15),
 ('잘', 15),
 ('다시', 15),
 ('2', 14),
 ('go', 14),
 ('night', 14),
 ('beautiful', 14),
 ('버벌진트', 14),
 ('so', 14),
 ('널', 14),
 ('넌', 14),
 ('안녕', 14),
 ('또', 14),
 ('bad', 13),
 ('crush', 13),
 (

In [33]:
words = df['album'].str.lower().str.cat(sep=' ')
word_tokens = word_tokenize(words)

word_counter = Counter(word_tokens)
print('{} different words'.format(len(word_counter)))
word_counter.most_common(538)

3093 different words


[('`', 571),
 ('part', 413),
 ('ost', 413),
 ('(', 361),
 ('the', 354),
 (')', 339),
 ('album', 269),
 ('-', 195),
 ('2', 154),
 ('1', 150),
 ('love', 116),
 ('mini', 110),
 ('4', 79),
 ('3', 73),
 ('1st', 70),
 ('.', 69),
 ('...', 68),
 ('1집', 68),
 ('you', 68),
 ('5', 67),
 ('of', 66),
 ('single', 66),
 (',', 61),
 ('me', 56),
 ('2nd', 55),
 ('a', 55),
 ('i', 55),
 ('가수다', 50),
 ('_', 48),
 ('ep', 48),
 ('나는', 47),
 ('s', 45),
 ('digital', 44),
 ('to', 43),
 ('episode', 43),
 ('경연', 42),
 ('soul', 41),
 ('&', 41),
 ('쇼미더머니', 40),
 ('3rd', 39),
 ('special', 38),
 ('#', 37),
 ('top', 37),
 ('6', 37),
 ('4th', 35),
 ('up', 34),
 ('2집', 34),
 ('my', 33),
 ('무한도전', 33),
 ('in', 32),
 ('new', 30),
 ('!', 29),
 ('11', 29),
 ('7', 27),
 ('for', 26),
 ('red', 26),
 ('it', 26),
 ('4집', 26),
 ('버스커', 26),
 ('first', 25),
 ('5th', 24),
 ('yourself', 24),
 ('가요제', 24),
 ('k팝', 23),
 ('스타', 23),
 ('5집', 23),
 ('6th', 23),
 ('?', 23),
 ('8', 23),
 ('t', 22),
 ('one', 21),
 ('겨울', 20),
 ('x', 20),
 

## Limpiar los nombres de artistas

### Múltiples artistas

In [34]:
df_feat = df[df.title.str.contains("feat")]
is_collab_index = df_feat.index
df_multiple_artist = df[df.singer.str.contains(r"(,|\s&\s)")]
is_collab_index = is_collab_index.union(df_multiple_artist.index)
df["is_collab"] = False
df.loc[is_collab_index, "is_collab"] = True

C:\Users\Paula\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Se planea utilizar el API de Spotify para conseguir la información pertinente de las canciones. A veces cuando se busca el nombre coreano en Spotify no se muestran resultados. Esto empeora cuando una canción tiene varios artistas. El objetivo será con la ayuda de DBkpop conseguir traducir los nombres coreanos a nombres en inglés o romanizados.

Para limpiar el campo `singer` se va a crear un dataframe para traducir el campo original con múltiples artistas a un campo con un solo artista.

In [35]:
a = df_multiple_artist[["singer"]].copy()
a.head(3)

,singer
49,"방탄소년단, zara larsson"
52,"스윙스(swings), 에일리"
57,"다이나믹 듀오(dynamic duo), dj premier"


El campo `singer` se separa mediante comas y se borran los valores que contengan nombres de artistas internacionales.

In [36]:
b = a.singer.str.split(r"\,", expand=True)[[0, 1, 2]]
b["original"] = a.singer
a = b
a[0] = a[0].str.strip()
a[1] = a[1].str.strip()
a[2] = a[2].str.strip()
a.drop_duplicates(keep='first', inplace=True)
a.loc[a[0].isin(international_singers), 0] = None
a.loc[a[1].isin(international_singers), 1] = None

In [37]:
a.head()

,0,1,2,original
49,방탄소년단,None,None,"방탄소년단, zara larsson"
52,스윙스(swings),에일리,None,"스윙스(swings), 에일리"
57,다이나믹 듀오(dynamic duo),dj premier,None,"다이나믹 듀오(dynamic duo), dj premier"
59,마마무 (mamamoo),에스나 (esna),None,"마마무 (mamamoo), 에스나 (esna)"
70,정은지,서인국,None,"정은지, 서인국"


El nuevo campo se llamará `new`. Algunos valores indican la presencia de múltiples artistas mediante el símbolo *&*. El nuevo valor será la primera parte de la coma siempre y cuando no tenga "&".

In [39]:
a["new"] = None
a["new"] = a[~a[0].isnull() & a[0].str.contains("&") == False][0]

Se exploran los valores que sí cuentan con el signo "&" y se encuentran casos especiales que se tratarán de forma manual.

In [40]:
a[~a[0].isnull() & a[0].str.contains("&")][0].unique()

array(['mc 몽 & 서인영', '이기찬 & g.na', 'g.a.b (길 & boa)', '15& (박지민',
       '택연 & 닉쿤', '하우두유둘 (유재석 & 유희열)', '슈프림팀 & 영준 (브라운아이드소울)',
       '이승기 & 김연아', '다이나믹 듀오 (dynamic duo) & 첸 (chen)', '타우 & 하하',
       '김범수 & 태연', '택연 & 우영 & 수지 & 김수현 & joo', '손승연 & 더 원 (the one)',
       '환희 & 숙희', 'f(x) (루나 & 크리스탈)', '병살 (정준하 & 김c)', '마이티마우스 & 백지영',
       '세븐티핑거스 (하하 & 장기하와 얼굴들)', '장미하관 (노홍철 & 장미여관)',
       '형용돈죵 (정형돈 & g-dragon)', '씨잼(cjamm) & 레디(reddy)'], dtype=object)

Se reemplazan algunos valores:

In [41]:
replace = {
           "15& (박지민": "15&", 
        'f(x) (루나 & 크리스탈)': 'f(x)',
        '다이나믹 듀오 (dynamic duo) & 첸 (chen)': '다이나믹 듀오(dynamic duo)',
        '씨잼(cjamm) & 레디(reddy)': 'cjamm'
          }
a.loc[~a[0].isnull(),"new"] = a[0].map(replace)

Se listan los valores con "&" después de haber cambiado algunos en la celda anterior. En estos valores se observa un patrón que se aprovecha mediante una expresión `regex` para quedarse con uno de los artistas.

Ejemplos:

- mc 몽 <span style="color: red">& 서인영</span>  ->  mc 몽 
- g.a.b <span style="color: red">(길 & boa)</span>  -> g.a.b

In [42]:
a[~a[0].isnull() & a[0].str.contains("&") & a['new'].isnull()][0].unique()

array(['mc 몽 & 서인영', '이기찬 & g.na', 'g.a.b (길 & boa)', '택연 & 닉쿤',
       '하우두유둘 (유재석 & 유희열)', '슈프림팀 & 영준 (브라운아이드소울)', '이승기 & 김연아',
       '타우 & 하하', '김범수 & 태연', '택연 & 우영 & 수지 & 김수현 & joo',
       '손승연 & 더 원 (the one)', '환희 & 숙희', '병살 (정준하 & 김c)', '마이티마우스 & 백지영',
       '세븐티핑거스 (하하 & 장기하와 얼굴들)', '장미하관 (노홍철 & 장미여관)',
       '형용돈죵 (정형돈 & g-dragon)'], dtype=object)

In [43]:
a.loc[~a[0].isnull() & a[0].str.contains("&") & a['new'].isnull(), "new"] = a[~a[0].isnull() & a[0].str.contains("&") & a['new'].isnull()][0].str.replace(r'(\(.+)?(&.+)*', '').str.strip() 

Todavía quedan filas en `new` que están vacías, por lo que se sigue con el proceso.

In [44]:
a[a["new"].isnull()]

,0,1,2,original,new
49,방탄소년단,None,None,"방탄소년단, zara larsson",NaN
52,스윙스(swings),에일리,None,"스윙스(swings), 에일리",NaN
57,다이나믹 듀오(dynamic duo),dj premier,None,"다이나믹 듀오(dynamic duo), dj premier",NaN
59,마마무 (mamamoo),에스나 (esna),None,"마마무 (mamamoo), 에스나 (esna)",NaN
70,정은지,서인국,None,"정은지, 서인국",NaN
...,...,...,...,...,...
3473,김효은,차붐,ek,"김효은, 차붐, ek",NaN
3512,케이윌 (k.will),소유,정민,"케이윌 (k.will), 소유, 정민",NaN
3562,케이윌 (k.will),전우성 (노을),None,"케이윌 (k.will), 전우성 (노을)",NaN
3575,김나영,양다일,None,"김나영, 양다일",NaN


Los valores que sean como `primer nombre & otros valores` se habrán convertido en `primer nombre`. El resto de valores se asignan tal cual al saber que son un solo artista.

In [45]:
a.loc[a["new"].isnull(), "new"] = a[0]

Se comprueba que valores quedan de asignar a `new` y solo es uno. Esto ha ocurrido porque *Dua Lipa* es una cantante estadounidense, por lo que se ha eliminado previamente, dejando el campo vacío.

In [46]:
a[a["new"].isnull()]

,0,1,2,original,new
855,None,화사(hwa sa),None,"dua lipa, 화사(hwa sa)",None


Si el campo `new` está vacío, el campo `0` está vacío y el campo `1` no lo está, el valor nuevo será el valor del campo `1`.

In [47]:
a.loc[a["new"].isnull() & a[0].isnull() & ~a[1].isnull(), "new"] = a[1]

No hay ningún campo que quede por limpiar.

In [48]:
a[a["new"].isnull()]

,0,1,2,original,new


El *dataset* con las traducciones finales para los artistas:

In [49]:
collaboration_mapping = a[["original", "new"]]
collaboration_mapping.head(5)

,original,new
49,"방탄소년단, zara larsson",방탄소년단
52,"스윙스(swings), 에일리",스윙스(swings)
57,"다이나믹 듀오(dynamic duo), dj premier",다이나믹 듀오(dynamic duo)
59,"마마무 (mamamoo), 에스나 (esna)",마마무 (mamamoo)
70,"정은지, 서인국",정은지


### Artista único

Ya tenemos las traducciones para convertir múltiples artistas en uno solo, ahora se intentará traducir y limpiar cada nombre.

Se borran los duplicados de los nombres de los artistas.

In [50]:
a = df.index.difference(df_multiple_artist.index)
a = df.loc[a]
a = a[["singer"]].copy()
a.drop_duplicates(keep='first', inplace=True)
a

,singer
0,볼빨간사춘기
1,키썸 (kisum)
2,에이핑크 (apink)
3,방탄소년단
4,유키스 (u-kiss)
...,...
3571,제아파이브(ze:a five)
3582,소찬휘
3585,보이비 (boi b)
3620,c-clown(씨클라운)


Se concatenan las listas de artistas que se tienen.

In [51]:
collaboration = collaboration_mapping.rename(columns={"new": "singer"})
a = pd.concat([collaboration[['singer']], a]).copy()
a.drop_duplicates(keep='first', inplace=True)
a.head(3)

,singer
49,방탄소년단
52,스윙스(swings)
57,다이나믹 듀오(dynamic duo)


Se juntan algunas de las tablas de DBKpop.

In [ ]:
group_names = pd.concat([df_girl[["name", "short", "korean_name", "gender"]], df_boy[["name", "short", "korean_name", "gender"]]])
singer_names = df_idols[["stage_name", "full_name", "korean_name", "k._stage_name"]]
singer_names.drop_duplicates(keep='first', inplace=True)

In [53]:
display(group_names.head(2))
display(singer_names.head(2))

,name,short,korean_name,gender
0,girls' generation,snsd,소녀시대,girlgroup
1,kara,NaN,카라,girlgroup


,stage_name,full_name,korean_name,k._stage_name
2,taeyeon,kim taeyeon,김태연,태연
3,sunny,lee sunkyu,이순규,써니


Tras ver los datos se llega a la conclusión de que en muchas ocasiones Gaon lo que hace es incluir dos nombres, el coreano y el romanizado o el inglés entre paréntesis. Los nombres de los cantantes se limpiarán teniendo esto en cuenta.

Los nombres de los cantantes se separan por paréntesis.

In [54]:
b = a.singer.str.split(r"\(", expand=True)
b["original"] = a.singer
b["new"] = None
a = b
a.head(3)

,0,1,2,original,new
49,방탄소년단,None,None,방탄소년단,None
52,스윙스,swings),None,스윙스(swings),None
57,다이나믹 듀오,dynamic duo),None,다이나믹 듀오(dynamic duo),None


In [55]:
a.shape

(907, 5)

In [56]:
a[~a[2].isnull()]

,0,1,2,original,new
2061,이유 갓지,god g)않은이유,박명수,이유 갓지(god g)않은이유(박명수,None
1241,크리스탈,f,x)),크리스탈 (f(x)),None


Hay algunos casos especiales que se cambian manualmente.

In [57]:
a.loc[a.original == "크리스탈 (f(x))", "new"] = "f(x)"
a.loc[a.original == "이유 갓지(god g)않은이유(박명수", "new"] = "god g"
a.drop(2, axis=1, inplace=True)

In [58]:
a[1] = a[1].str.replace(")", "")

In [59]:
a.head(3)

,0,1,original,new
49,방탄소년단,None,방탄소년단,None
52,스윙스,swings,스윙스(swings),None
57,다이나믹 듀오,dynamic duo,다이나믹 듀오(dynamic duo),None


In [60]:
a.drop_duplicates(keep='first', inplace=True)

In [61]:
a[0] = a[0].str.strip()
a[1] = a[1].str.strip()

In [62]:
display(group_names.head(2))
display(singer_names.head(2))

,name,short,korean_name,gender
0,girls' generation,snsd,소녀시대,girlgroup
1,kara,NaN,카라,girlgroup


,stage_name,full_name,korean_name,k._stage_name
2,taeyeon,kim taeyeon,김태연,태연
3,sunny,lee sunkyu,이순규,써니


In [63]:
a.shape

(907, 4)

Se va a utilizar el *dataset* de DBKpop de los grupos para conseguir el romanizado o nombre en inglés de los grupos.

In [64]:
b = a.copy()
for l in [0,1]:
    for r in ["name", "short", "korean_name"]:        
        b.loc[b[l].isnull(), l] = 999999
        b = pd.merge(b, group_names, left_on=l, right_on=r, how="left").drop(['short', "korean_name", "gender"], 
                                                                                    axis=1)
        
        b.loc[~b.name.isnull(), "new"] = b.name
        b.drop("name", axis=1, inplace=True)
        b.loc[b[l] == 999999, l] = None
a = b

In [65]:
a.shape

(907, 4)

Se iba a utilizar el *dataset* de DBKpop de los artistas para conseguir el romanizado o nombre en inglés pero hay idols coreanos que tienen el mismo `k._stage_name` y `nombre` por lo que esto no es posible porque puede llevar a errores.

In [66]:
singer_names[singer_names.duplicated('k._stage_name')].head(5)

,stage_name,full_name,korean_name,k._stage_name
34,minah,bang minah,방민아,민아
78,yeeeun,jang yeeun,장예은,예은
79,hyuna,moon hyuna,문현아,현아
85,sojin,jo sojin,조소진,소진
112,dasom,lee dasom,이다솜,다솜


In [67]:
singer_names[singer_names.duplicated('korean_name')].head(5)

,stage_name,full_name,korean_name,k._stage_name
155,hyomin,park sunyoung,박선영,효민
163,chahee,park sooyoung,박수영,차희
178,lizzy,park sooyoung,박수영,리지
208,bona,kim jiyeon,김지연,보나
214,seola,kim hyunjung,김현정,설아


In [68]:
singer_names[singer_names.duplicated('korean_name') & singer_names.duplicated('k._stage_name')].head(5)

,stage_name,full_name,korean_name,k._stage_name
314,jimin,park jimin,박지민,지민
557,subin,park subin,박수빈,수빈
732,jeewon,kim jiwon,김지원,지원
827,seunghyun,lee seunghyun,이승현,승현
854,juyeon,lee juyeon,이주연,주연


In [69]:
singer_names[singer_names['k._stage_name'] == "민혁"]

,stage_name,full_name,korean_name,k._stage_name
343,minhyuk,lee minhyuk,이민혁,민혁
533,minhyuk,kang minhyuk,강민혁,민혁
918,minhyuk,kim minhyuk,김민혁,민혁


Por este motivo, se elegirá la columna `0` o `1` en función de si se encuentra dicho valor en las tablas DBKPop de las anteriores celdas. Si el nombre no se ha encontrado...
- si la columna `1` está vacía se elige la `0` por defecto.
- si se encuentran letras (a,b,...) se prioriza la columna correspondiente, ya que se prefiere un nombre en inglés o romanizado que en coreano.

In [70]:
for c in singer_names.columns:
     a.loc[~a.new.isnull() & a[1].isin(singer_names.loc[~singer_names[c].isnull(),c]), "new"] = a[1]
     a.loc[~a.new.isnull() & a[0].isin(singer_names.loc[~singer_names[c].isnull(),c]), "new"] = a[0]

In [71]:
a.loc[a.new.isnull() & a[1].isnull(), "new"] = a[0]

In [72]:
a.loc[a.new.isnull() & a[0].str.match(r'[a-z]'), "new"] = a[0]

In [73]:
a.loc[a.new.isnull() & a[1].str.match(r'[a-z]'), "new"] = a[1]

Hay algunos nombres que no han cumplido con lo anterior porque tenían números (4men), se aplica otra regla mediante regex para detectar estos.

In [74]:
a.loc[a.new.isnull()]

,0,1,original,new
30,포맨,4men,포맨 (4men),None
35,거머리,박명수,거머리 (박명수,None
38,댄싱게놈,유재석,댄싱게놈(유재석,None
51,상주나,정준하,상주나(정준하,None
89,싹쓰리,유두래곤,싹쓰리 (유두래곤,None
125,황태지,황광희,황태지(황광희,None
127,오대천왕,정형돈,오대천왕(정형돈,None
165,으뜨거따시,하하,으뜨거따시(하하,None
177,이진성,먼데이키즈,이진성 (먼데이키즈),None
195,보람,씨야,보람 (씨야),None


In [75]:
a.loc[a.new.isnull() & a[1].str.match(r'\d\s?[a-z]+'), "new"] = a[1]

Los artistas que no han cumplido con las reglas anteriores son:

In [76]:
a.loc[a.new.isnull()]

,0,1,original,new
35,거머리,박명수,거머리 (박명수,None
38,댄싱게놈,유재석,댄싱게놈(유재석,None
51,상주나,정준하,상주나(정준하,None
89,싹쓰리,유두래곤,싹쓰리 (유두래곤,None
125,황태지,황광희,황태지(황광희,None
127,오대천왕,정형돈,오대천왕(정형돈,None
165,으뜨거따시,하하,으뜨거따시(하하,None
177,이진성,먼데이키즈,이진성 (먼데이키즈),None
195,보람,씨야,보람 (씨야),None
261,타블로,에픽하이,타블로 (에픽하이),None


In [77]:
a.loc[~a[1].isnull() & a[1].str.contains("gun"), "new"] = "#gun"

Se elige la columna `0` como el nuevo nombre.

In [78]:
a.loc[a.new.isnull(), "new"] = a[0]

In [79]:
a.loc[a.new.isnull()]

,0,1,original,new


El *dataset* de traducción para los artistas individuales será:

In [80]:
final_mapping = a.drop([0, 1], axis=1)
final_mapping.head(3)

,original,new
0,방탄소년단,bts
1,스윙스(swings),swings
2,다이나믹 듀오(dynamic duo),dynamic duo


El *dataset* de traducción para convertir múltiples artistas en uno solo:

In [81]:
collaboration_mapping.head(3)

,original,new
49,"방탄소년단, zara larsson",방탄소년단
52,"스윙스(swings), 에일리",스윙스(swings)
57,"다이나믹 듀오(dynamic duo), dj premier",다이나믹 듀오(dynamic duo)


Primero se utilizará `collaboration_mapping` en los valores "singer" que tengan múltiples artistas y luego se usará `final_mapping` sobre el mismo campo.

### Limpiar los artistas de Gaon

In [82]:
df2 = pd.merge(df, collaboration_mapping, left_on='singer', right_on='original', how="left")

In [83]:
df2.loc[df2.new.isnull(), "new"] = df2.singer

In [84]:
df2.drop('original', axis=1, inplace=True)

In [85]:
df2 = pd.merge(df2, final_mapping, left_on='new', right_on='original', how="left")
df2.drop(['new_x', 'original'], axis=1, inplace=True)
df2 = df2.rename(columns={"new_y": "clean_singer"})
df2.head(3)

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,num_times_top_1,is_collab,clean_singer
0,#첫사랑,볼빨간사춘기,#첫사랑,"쇼파르뮤직, 딩고뮤직",카카오 m,201801-201802-201803,1,0,1,1,0,False,bolbbalgan4
1,#화이트 스타일 (feat. 김호연 of 달 좋은밤),키썸 (kisum),show off,스케치,카카오 m,201507,0,0,0,0,0,True,kisum
2,%%(응응),에이핑크 (apink),percent,플레이엠엔터테인먼트,카카오 m,201901-201902,1,0,0,0,0,False,apink


In [86]:
df2.shape

(3502, 13)

Una vez que se han limpiado y traducido los nombres de los artistas se va a utilizar el API de Spotify para intentar obtener el spotify id correspondiente para posteriormente obtener sus detalles.

## Find songs in Spotify

In [32]:
SPOTIPY_CLIENT_ID='xxxx'
SPOTIPY_CLIENT_SECRET='xxxx'

#https://developer.spotify.com/dashboard/applications/8269427507e44ca4ba3cbd414a76144a

In [33]:
auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Spotify api
cont_si = 0
for index, row in df2.iterrows():
    title = row['title']
    singer = row['clean_singer']
    search=f'{title} artist:{singer}'
    print(index,"/",df2.shape[0]-1)
    track_results = sp.search(q=search, type='track', market='SK', limit=1)
    result_num = track_results['tracks']['total']
    if result_num > 0:
        cont_si += 1
        
    if result_num == 0:
        track_results = sp.search(q=search, type='track', limit=1)
        result_num = track_results['tracks']['total']
        if result_num > 0:
            cont_si += 1
        
        if result_num == 0:
            search=f'{title} {singer}'
            track_results = sp.search(q=search, type='track', limit=1)
            result_num = track_results['tracks']['total']
            if result_num > 0:
                cont_si += 1

    porcentaje = cont_si * 100 / df2.shape[0]
    print(porcentaje, "%")
    print(cont_si,"/",df2.shape[0])
    print()
    for i, t in enumerate(track_results['tracks']['items']):
        df2.loc[index,'spotify_uri'] = t['uri']

In [652]:
# Google search
import requests 
import bs4
from bs4 import BeautifulSoup
import urllib.parse

In [674]:
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

def google(q):
    spotify_id = ""
    s = requests.Session()
    q = urllib.parse.quote_plus(q)
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get)
    if r.status_code != 200:
        raise Exception(f"Error scrapping google: {r.text}, {r.status_code}")
    soup = BeautifulSoup(r.text, "html.parser")
    output = []
    soup = soup.find_all('td', {'class': ['ellip','YMAGHf']})
    if soup:
        soup = soup[0]
        url = soup.find('a')["href"] 
        if "open.spotify.com/track/" in url:
            spotify_id = f"spotify:track:{url.split('/track/')[1]}"
            if "?autoplay" in spotify_id:
                spotify_id = spotify_id.split("?autoplay")[0]
        
    return spotify_id

In [ ]:
df_uri_null = df2[df2.spotify_uri.isnull()]
cont = 1
cont_si = 0
for index, row in df_uri_null.iterrows():
    title = row['title'].strip()
    singer = row['singer'].strip()
    search= title + " " + singer
    print(cont,"/",df_uri_null.shape[0])
   
    spotify_id = google(search)
    if spotify_id:
        cont_si += 1
        df2.loc[index, 'spotify_uri'] = spotify_id
        
    porcentaje = cont_si * 100 / df_uri_null.shape[0]
    print(porcentaje, "%")
    print(cont_si,"/",df_uri_null.shape[0])
    print()
    cont += 1

También se ha utilizado el campo limpio del cantante para buscar el grupo en Google.

In [ ]:
df_uri_null = df2[df2.spotify_uri.isnull()]
cont = 1
cont_si = 0
for index, row in df_uri_null.iterrows():
    title = row['title'].strip()
    singer = row['clean_singer'].strip()
    search= title + " " + singer
    print(cont,"/",df_uri_null.shape[0])
   
    spotify_id = google(search)
    if spotify_id:
        cont_si += 1
        df2.loc[index, 'spotify_uri'] = spotify_id
        
    porcentaje = cont_si * 100 / df_uri_null.shape[0]
    print(porcentaje, "%")
    print(cont_si,"/",df_uri_null.shape[0])
    print()
    cont += 1

Habrá canciones que no se hayan encontrado por lo que se revisarán manualmente y se introducirán los identificadores de Spotify correspondientes. 

El 67% de las canciones no se han encontrado de forma automática.

In [433]:
df2[df2.spotify_uri.isnull()].shape

(2367, 4)

In [443]:
round(100/df2.shape[0]*(df2[df2.spotify_uri.isnull()].shape[0]),1)

67.6

In [705]:
df2.to_csv("tmp/songs.csv", sep=";", index=False)

Tras haber introducido manualmente los enlaces de Spotify faltantes y haber borrado aquellas canciones que no se encontraban en Spotify se comprueba el tamaño de los datos.

In [3]:
df2 = pd.read_csv("tmp/songs.csv", sep=";")

In [ ]:
df2.drop(['clean_singer'], axis=1, inplace=True)

In [451]:
df2.to_csv("tmp/songs.csv", sep=";", index=False)

In [89]:
df2.head(2)

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,num_times_top_1,is_collab,spotify_uri
0,#첫사랑,볼빨간사춘기,#첫사랑,"쇼파르뮤직, 딩고뮤직",카카오 m,201801-201802-201803,1,0,1,1,0,False,spotify:track:4tKN17Kw29gzlnZwMaAtZN
1,#화이트 스타일 (feat. 김호연 of 달 좋은밤),키썸 (kisum),show off,스케치,카카오 m,201507,0,0,0,0,0,True,spotify:track:1LrcLQ3pTFz7yafs0a2sds


In [71]:
df2.shape

(3070, 13)

In [27]:
df2[df2.spotify_uri.isnull()].shape

(0, 13)

Se comprueba que no haya duplicados.

In [90]:
duplicated = df2[df2.duplicated()]
duplicated

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,num_times_top_1,is_collab,spotify_uri


In [91]:
duplicated_spotify_uri = df2[df2.spotify_uri.duplicated()]
duplicated_spotify_uri

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,num_times_top_1,is_collab,spotify_uri


In [92]:
df2[df2[['title', 'singer']].duplicated()]

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,num_times_top_1,is_collab,spotify_uri


----------------------------------------

## Spotify API

Una vez que se tienen las canciones con el link correspondiente de Spotify, se procede a utilizar el servicio _Spotify API_ para ampliar los datos de los que se dispone.

La API de _Spotify_ especifica que solo se pueden obtener 50 canciones a la vez, por lo que se obtendrá la información de 50 en 50.

In [164]:
df2.spotify_uri = df2.spotify_uri.str.replace("spotify:track:", "")
df2.spotify_uri.head()

0    4tKN17Kw29gzlnZwMaAtZN
1    1LrcLQ3pTFz7yafs0a2sds
2    4SpQDJz1JiRT2mHxvyqN9K
3    4AlihYDqxXshKhvh5tnMfP
4    7HKDOS7RKlnUkluAY23oga
Name: spotify_uri, dtype: object

In [72]:
num_songs = df2.shape[0]
num_songs

3070

In [34]:
def extract_songs_basic_info(basic_info, track_info):
    # https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-tracks/
    singers = "-sep-".join([x['name'].lower() for x in track_info["artists"]])
    singers_id = "-sep-".join([x['id'] for x in track_info["artists"]])
    basic_info['singer'].append(singers)
    basic_info['singer_id'].append(singers_id)
    basic_info['album_type'].append(track_info["album"]["album_type"])
    basic_info['release_date'].append(track_info["album"]["release_date"])
    basic_info['duration_ms'].append(track_info["duration_ms"])
    basic_info['explicit'].append(track_info["explicit"])
    
    # check size of dict
    size = len(basic_info["album_type"])
    if len(basic_info["release_date"]) != size or \
                                                    len(basic_info["duration_ms"]) != size or \
                                                    len(basic_info["explicit"]) != size or \
                                                    len(basic_info["singer"]) != size or \
                                                    len(basic_info["singer_id"]) != size:
        raise Exception(f"Distinct size for {track_info}")
        

def extract_songs_audio_info(audio_info, track_info):
    # https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/
    size = None
    for k,v in track_info.items():
        if k in audio_info.keys():
            audio_info[k].append(v)
            current_size = len(audio_info[k])
            if not size:
                size = current_size
            else:
                if size != current_size:
                    raise Exception(f"Distinct size for {track_info}")

# get song info
basic_info = {
    # basic
    "singer": [],
    "singer_id": [],
    "album_type":  [],
    "release_date": [],
    "duration_ms": [],
    "explicit": [],
}

audio_info = {
    # audio features
    "acousticness": [],
    "danceability": [],
    "energy": [],
    "instrumentalness": [],
    "liveness": [],
    "loudness": [],
    "speechiness": [],
    "valence": [],
    "tempo": []
}
            
# SPOTIFY API
for i in list(range(50, num_songs+50, 50)):
    # conseguir información de las canciones de 50 en 50
    to_row = min(i, num_songs)
    from_row = i-50    
    print(f"starting  from {from_row} to {to_row}...")
    
    batch_ids = df2.spotify_uri[from_row:to_row].to_list()
    
    print("get songs basic info")
    tracks_info = sp.tracks(batch_ids) # max 50
    print("extract basic info")
    list(map(lambda t:extract_songs_basic_info(basic_info, t), tracks_info["tracks"]))
    
    print("get songs audio features")
    tracks_info = sp.audio_features(batch_ids) # max 50
    print("extract audio info")
    list(map(lambda t:extract_songs_audio_info(audio_info, t), tracks_info))
    
    print("#"*50)
    
print("adding info to dataframe")
# add info to dataframe
info = {**basic_info, **audio_info}
for k,v in info.items():
    df2[k] = pd.Series(v)

starting  from 0 to 50...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 50 to 100...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 100 to 150...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 150 to 200...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 200 to 250...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 250 to 300...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 300 t

extract audio info
##################################################
starting  from 2550 to 2600...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 2600 to 2650...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 2650 to 2700...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 2700 to 2750...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 2750 to 2800...
get songs basic info
extract basic info
get songs audio features
extract audio info
##################################################
starting  from 2800 to 2850...
get songs basic info
extract basic info
get songs audio features
extr

In [103]:
df2.to_csv("tmp/songs.csv", index=False, sep=";")

## Atributos

In [74]:
df2.columns

Index(['title', 'singer', 'album', 'producer', 'distributor', 'year_month',
       'is_popular', 'external_factors', 'num_times_top_10', 'num_times_top_5',
       'num_times_top_1', 'is_collab', 'spotify_uri', 'album_type',
       'release_date', 'duration_ms', 'explicit', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'valence', 'tempo', 'singer_id', 'is_ost'],
      dtype='object')

In [75]:
df2.head().T

,0,1,2,3,4
title,#첫사랑,#화이트 스타일 (feat. 김호연 of 달 좋은밤),%%(응응),00_00 (zero o`clock),0330
singer,bol4,kisum,apink,bts,u-kiss
album,#첫사랑,show off,percent,map of the soul _ 7,bran new kiss (ep)
producer,"쇼파르뮤직, 딩고뮤직",스케치,플레이엠엔터테인먼트,빅히트 엔터테인먼트,nh emg
distributor,카카오 m,카카오 m,카카오 m,dreamus,카카오 m
year_month,201801-201802-201803,201507,201901-201902,202003-202004-202005,201105
is_popular,1,0,1,1,0
external_factors,0,0,0,0,0
num_times_top_10,1,0,0,0,0
num_times_top_5,1,0,0,0,0


Algunas comprobaciones...

In [76]:
df2.query('(num_times_top_10>1 or num_times_top_5>1 or num_times_top_1>1) and is_popular==0')

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,singer_id,is_ost


In [77]:
df2.query('external_factors== 1 and is_popular==0')

,title,singer,album,producer,distributor,year_month,is_popular,external_factors,num_times_top_10,num_times_top_5,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,singer_id,is_ost


### Feat

Este atributo ya se ha creado anteriormente. Las canciones tendrán el atributo `is_collab` como True si:
- Feat o duet en título (en parte realizado en un apartado previo)
- “,” o “ & “ en artista (ya realizado en un apartado previo)

In [78]:
df2.loc[df2.title.str.contains("duet"), "is_collab"] = True

### Album type

In [4]:
df2.album_type.value_counts()

single         1605
album          1252
compilation     215
Name: album_type, dtype: int64

### OST

In [105]:
is_ost_index = df2[df2.album.str.contains(r'(.\s|^)ost(\s.|$)')].index
is_ost_index = is_ost_index.union(df2[df2.title.str.contains(r'(.\s|^)ost(\s.|$)')].index)
df2.iloc[is_ost_index][["title", "singer", "album"]].head(3)

C:\Users\Paula\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,title,singer,album
43,a brand new day (bts world ost part.2),bts-sep-zara larsson,a brand new day (bts world ost part.2)
52,always,yoon mirae,태양의 후예 ost part 1
63,and i`m here,kim kyung hee,도깨비 ost part 11


In [106]:
df2['is_ost'] = 0
df2.loc[is_ost_index, "is_ost"] = 1

### Tv show

In [30]:
reality_tv_shows = ["k팝 스타", "슈퍼스타k", "쇼미더머니", "무한도전",  
                    "언프리티 랩스타", "고등래퍼", "나는 가수다", "복면가왕", "produce"]

In [35]:
df2["for_reality_tvshow"] = 0
for r in reality_tv_shows:
    df2.loc[df2.album.str.contains(r), "for_reality_tvshow"] = 1

In [40]:
df2.to_csv("tmp/songs.csv", sep=";", index=False)